In [37]:
import os
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
from shapely.geometry import Point
from pathlib import Path
import re
from rasterio.mask import mask
from shapely.geometry import box
from fiona.crs import from_epsg
import json
#import rioxarray as xr

In [9]:
ls tif_data_2021/LC08_L2SP_022031_20210514_20210525_02_T1

LC08_L2SP_022031_20210514_20210525_02_T1_ANG.txt
LC08_L2SP_022031_20210514_20210525_02_T1_MTL.txt
LC08_L2SP_022031_20210514_20210525_02_T1_MTL.xml
LC08_L2SP_022031_20210514_20210525_02_T1_SR_B1.TIF
LC08_L2SP_022031_20210514_20210525_02_T1_SR_B2.TIF
LC08_L2SP_022031_20210514_20210525_02_T1_SR_B3.TIF
LC08_L2SP_022031_20210514_20210525_02_T1_SR_B4.TIF
LC08_L2SP_022031_20210514_20210525_02_T1_SR_B5.TIF
LC08_L2SP_022031_20210514_20210525_02_T1_SR_B6.TIF
LC08_L2SP_022031_20210514_20210525_02_T1_SR_B7.TIF
LC08_L2SP_022031_20210514_20210525_02_T1_ST_B10.TIF


In [11]:
#File path
SHARED_DATA_FOLDER = Path('tif_data_2021')
#construct SCENE NAME
# path_row = []  
# year = []
# month = []
# day = []
SCENE = 'LC08_L2SP_022031_20210514_20210525_02_T1'
#check if scene_path is a file

In [12]:
scene_path = SHARED_DATA_FOLDER/SCENE
b1_path =scene_path/"{}_SR_B2.TIF".format(SCENE)
b2_path =scene_path/"{}_SR_B2.TIF".format(SCENE)
b3_path =scene_path/"{}_SR_B3.TIF".format(SCENE)
b4_path =scene_path/"{}_SR_B4.TIF".format(SCENE)
b5_path =scene_path/"{}_SR_B5.TIF".format(SCENE)
b6_path =scene_path/"{}_SR_B6.TIF".format(SCENE)
b7_path =scene_path/"{}_SR_B7.TIF".format(SCENE)
b10_path = scene_path/"{}_ST_B10.TIF".format(SCENE)

In [13]:
b1 = rasterio.open(b1_path).read(1)
# b2 = rasterio.open(b2_path).read(1)
# b3 = rasterio.open(b3_path).read(1)
# b4 = rasterio.open(b4_path).read(1)
# b5 = rasterio.open(b5_path).read(1)
# b6 = rasterio.open(b6_path).read(1)
# b7 = rasterio.open(b7_path).read(1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [34]:
data = rasterio.open(b1_path)
c = data.crs
c

CRS.from_epsg(32616)

In [17]:
#Get CHI BB:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [22]:
chi_b = gpd.read_file("chi_b").to_crs(raster.crs)
chi_b

,id,geometry
0,None,"POLYGON ((421529.030 4645627.453, 421709.344 4..."


In [43]:
def clip_raster(raster, vector_poly):
    '''
    Clip raster polygon with the vecotr polygon boundary
    Input: 
        raster - rasterio object (i.e. read from rasterio.open)
        vector_poly - geopandas dataframe
    Output:
        raster object that is clipped
    '''
    vector_poly_good_crs = vector_poly.to_crs(raster.crs)
    vector_as_json = [json.loads(vector_poly.to_json())['features'][0]['geometry']]
    
    out_img, out_transform = mask(dataset=raster, 
                                  shapes=vector_as_json, 
                                  crop=True)
    return out_img
    
    

In [44]:
new_raster = clip_raster(data, chi_b)

In [45]:
new_raster

array([[[    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        ...,
        [    0,     0,     0, ..., 10508, 10118, 10047],
        [    0,     0,     0, ..., 10394, 10340, 10008],
        [    0,     0,     0, ..., 10294, 10444, 10315]]], dtype=uint16)

In [24]:
chi_b_json = getFeatures(chi_b)
chi_b_json

[{'type': 'Polygon',
  'coordinates': [[[421529.0299251836, 4645627.452825234],
    [421709.34371174604, 4653024.683823592],
    [445652.14019487315, 4653593.279869158],
    [449597.58601560234, 4640484.426552334],
    [456798.8229183247, 4623790.817900849],
    [457114.65979003883, 4610445.840924643],
    [434622.36365102226, 4610747.52882748],
    [432215.6041096891, 4639043.377106867],
    [421529.0299251836, 4645627.452825234]]]}]

In [26]:
out_img, out_transform = mask(dataset=data, shapes=chi_b_json, crop=True)

In [29]:
out_img.shape

(1, 1439, 1187)

In [11]:
clipped_xarray = raster.rio.clip(chi_b)

AttributeError: 'DatasetReader' object has no attribute 'rio'

In [20]:
raster = rasterio.open(b1_path)
array_dtype = np.dtype('uint16')
i,j = np.indices(b1.shape, array_dtype)
longitude, latitude = raster.transform * (j, i)

In [5]:
pattern = '^(?:[^_]+_){3}([^_ ]+)'
period_str= re.findall(pattern, SCENE)[0]

In [8]:
ndsi =np.where((b3+b6)==0, 0, (b3-b6)/(b3+b6))
albedo = ((0.356*b1)+(0.1310*b2)+(0.373*b3)+(0.085*b4)+(0.072*b5)-0.0018)/1.016
awei = 4*(b3-b6)-(0.25*b5 + 2.75*b6)
ndvi = np.where((b4+b5)==0, 0, (b5-b4)/(b5+b4))
ndbi = np.where((b6+b5)==0, 0, (b6-b5)/(b6+b5))
eta = (2*(b5**2-b4**2) + 1.5*b5 + 0.5*b4) / (b5+b4+0.5)
gemi = eta*(1-0.25*eta) - ((b4-0.125)/(1-b4))

<ipython-input-8-45c57a3d1d06>:1: RuntimeWarning: divide by zero encountered in true_divide
  ndsi =np.where((b3+b6)==0, 0, (b3-b6)/(b3+b6))
<ipython-input-8-45c57a3d1d06>:1: RuntimeWarning: invalid value encountered in true_divide
  ndsi =np.where((b3+b6)==0, 0, (b3-b6)/(b3+b6))
<ipython-input-8-45c57a3d1d06>:4: RuntimeWarning: divide by zero encountered in true_divide
  ndvi = np.where((b4+b5)==0, 0, (b5-b4)/(b5+b4))
<ipython-input-8-45c57a3d1d06>:4: RuntimeWarning: invalid value encountered in true_divide
  ndvi = np.where((b4+b5)==0, 0, (b5-b4)/(b5+b4))
<ipython-input-8-45c57a3d1d06>:5: RuntimeWarning: divide by zero encountered in true_divide
  ndbi = np.where((b6+b5)==0, 0, (b6-b5)/(b6+b5))
<ipython-input-8-45c57a3d1d06>:5: RuntimeWarning: invalid value encountered in true_divide
  ndbi = np.where((b6+b5)==0, 0, (b6-b5)/(b6+b5))
<ipython-input-8-45c57a3d1d06>:7: RuntimeWarning: divide by zero encountered in true_divide
  gemi = eta*(1-0.25*eta) - ((b4-0.125)/(1-b4))


In [9]:
columns = ['ndsi', 'albedo', 'awei', 'ndbi', 'ndvi', 'gemi', 'latitude', 'longitude']
data = {}
features = [ndsi, albedo, awei, ndbi, ndvi, gemi, latitude, longitude]
for i, feature in enumerate(features):
    key = columns[i]
    data[key] = np.ndarray.flatten(feature)

In [11]:
landsat = pd.DataFrame(data)
landsat['period'] = period_str
landsat['period'] = pd.to_datetime(landsat['period'],yearfirst=True, format ='%Y%m%d')

In [19]:
landsat.shape

(61008121, 9)

In [14]:
landsat.describe()

,ndsi,albedo,awei,ndbi,ndvi,gemi,latitude,longitude
count,6.100812e+07,6.100812e+07,6.100812e+07,6.100812e+07,6.100812e+07,6.100812e+07,6.100812e+07,6.100812e+07
mean,1.262766e+00,1.909222e+04,-4.283094e+03,1.057271e+00,2.950885e+00,-inf,4.622565e+06,5.158350e+05
std,6.503644e+01,1.937312e+04,2.160685e+04,5.894265e+01,1.331509e+02,NaN,6.816486e+04,6.712563e+04
min,0.000000e+00,-1.771654e-03,-1.963620e+05,0.000000e+00,0.000000e+00,-inf,4.504515e+06,3.995850e+05
25%,0.000000e+00,-1.771654e-03,-2.069775e+04,0.000000e+00,0.000000e+00,-2.983274e+00,4.563525e+06,4.576950e+05
50%,3.903613e-01,9.336494e+03,0.000000e+00,5.346356e-01,1.186387e-01,1.250000e-01,4.622565e+06,5.158350e+05
75%,6.027409e-01,4.065191e+04,3.569750e+03,8.892068e-01,2.368377e+00,1.678326e-01,4.681605e+06,5.739750e+05
max,6.147500e+04,6.551842e+04,5.127300e+04,6.306100e+04,6.552500e+04,1.125000e+00,4.740615e+06,6.320850e+05


In [20]:
landsat

,ndsi,albedo,awei,ndbi,ndvi,gemi,latitude,longitude,period
0,0.0,-0.001772,0.0,0.0,0.0,0.125,4740615.0,399585.0,2014-01-03
1,0.0,-0.001772,0.0,0.0,0.0,0.125,4740615.0,399615.0,2014-01-03
2,0.0,-0.001772,0.0,0.0,0.0,0.125,4740615.0,399645.0,2014-01-03
3,0.0,-0.001772,0.0,0.0,0.0,0.125,4740615.0,399675.0,2014-01-03
4,0.0,-0.001772,0.0,0.0,0.0,0.125,4740615.0,399705.0,2014-01-03
...,...,...,...,...,...,...,...,...,...
61008116,0.0,-0.001772,0.0,0.0,0.0,0.125,4504515.0,631965.0,2014-01-03
61008117,0.0,-0.001772,0.0,0.0,0.0,0.125,4504515.0,631995.0,2014-01-03
61008118,0.0,-0.001772,0.0,0.0,0.0,0.125,4504515.0,632025.0,2014-01-03
61008119,0.0,-0.001772,0.0,0.0,0.0,0.125,4504515.0,632055.0,2014-01-03


In [21]:
landsat.to_csv("sample_df")

KeyboardInterrupt: 